In [1]:
!pip install diffusers
!pip install transformers
!pip install scipy
!pip install ftfy
!pip install accelerate
!pip install modin[all]
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.4 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 6.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 6.7 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 257.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 603.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.6 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.4/980.4 kB 6.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
import torch #needed only for GPU
from PIL import Image
from io import BytesIO
import numpy as np
from diffusers import StableDiffusionLatentUpscalePipeline, StableDiffusionUpscalePipeline
import gradio as gr
import modin.pandas as pd

# load model for CPU or GPU

model_2x = "stabilityai/sd-x2-latent-upscaler"
model_4x = "stabilityai/stable-diffusion-x4-upscaler"
device = "cuda" if torch.cuda.is_available() else "cpu"
upscaler2x = StableDiffusionLatentUpscalePipeline.from_pretrained(model_2x, torch_dtype=torch.float16) if torch.cuda.is_available() else StableDiffusionLatentUpscalePipeline.from_pretrained(model_2x)
upscaler4x = StableDiffusionUpscalePipeline.from_pretrained(model_4x, torch_dtype=torch.float16, revision="fp16") if torch.cuda.is_available() else StableDiffusionUpscalePipeline.from_pretrained(model_4x)
upscaler2x = upscaler2x.to(device)
upscaler4x = upscaler4x.to(device)

#define interface 

def upscale(raw_img, model, prompt, negative_prompt, scale, steps, Seed):
    generator = torch.manual_seed(Seed)
    raw_img = Image.open(raw_img).convert("RGB")
    if model == "Upscaler 4x":
        low_res_img = raw_img.resize((128, 128))
        upscaled_image = upscaler4x(prompt=prompt, negative_prompt=negative_prompt, image=low_res_img, guidance_scale=scale, num_inference_steps=steps).images[0]
    else:
        raw_img = raw_img.resize((256, 176)) 
        upscaled_image = upscaler2x(prompt=prompt, negative_prompt=negative_prompt, image=raw_img, guidance_scale=scale, num_inference_steps=steps).images[0]
    return upscaled_image
    
#launch interface
    
# gr.Interface(fn=upscale, inputs=[
#     gr.Image(type="filepath", label='Lower Resolution Image'), 
#     gr.Radio(['Upscaler 2x','Upscaler 4x'], label="Models"),
#     gr.Textbox(label="Optional: Enter a Prompt to Guide the AI's Enhancement, this can have an Img2Img Effect"), 
#     gr.Textbox(label='Experimental: Influence What you do not want the AI to Enhance. Such as Blur, Smudges, or Pixels'), 
#     gr.Slider(1, 15, 1, step=1, label='Guidance Scale: How much the AI influences the Upscaling.'), 
#     gr.Slider(5, 50, 5, step=1, label='Number of Iterations'),
#     gr.Slider(minimum=1, maximum=999999999999999999, randomize=True, step=1)], 
#     outputs=gr.Image(type="filepath", label = 'Upscaled Image'), 
#     title='SD Upscaler', 
#     description='2x Latent Upscaler using SD 2.0 And 4x Upscaler using SD 2.1. This version runs on CPU or GPU and is currently running on a T4 GPU. For 4x Upscaling use images lower than 512x512, ideally 128x128 or smaller for 512x512 output. For 2x Upscaling use up to 512x512 images for 1024x1024 output.<br><br><b>Notice: Largest Accepted Resolution is 512x512', 
#     article = "Code Monkey: <a href=\"https://huggingface.co/Manjushri\">Manjushri</a>").launch(share=True)


vae/diffusion_pytorch_model.safetensors not found


In [33]:
import os

In [34]:
bad_path ='/home/timssh/ML/TAGGING/test/bad/'
os.listdir(bad_path)

['b9de88cab8d2112cb7e92600fdadb730.jpg',
 '069c4a58dc666a49b5095f366cdec20e.jpg',
 'e7bf1cc487ccce683fe2c17436c15c2d.jpg',
 'a266eee9b76108a115b52c4a3ade797e.jpg',
 '9ae3811cc4c01b61f1828fa10e59b36a.jpg',
 'd6f73460e57232ee675f5af8ff869cc7.jpg',
 'c85f42eb925a433e66cca407eee039df.jpg',
 '1d202ecb6781efee8d88889d81734745.jpg',
 'ec6105c9d85302d90f2fcbb8abea6016.jpg',
 'd426e685c59df3bff996762ba1197427.jpg',
 '0fb7ddbd49b10ffb2555d1663df95051.jpg',
 'ce1ce86850cdc417a81fee946e445972.jpg',
 '33ac8b5ec173216b02780f4de877549f.jpg',
 'dd9d1c99f4cec57f7823b3094522ce69.jpg']

In [ ]:
for path in os.listdir(bad_path):
    raw_img = os.path.join(bad_path, path)
    upscale(raw_img, "Upscaler 2x", '', '', 5, 10, 34566).save(os.path.join('flagged', path))



In [38]:
raw_img = os.path.join(bad_path, path)
img = Image.open(raw_img).convert("RGB")
# low_res_img = raw_img.resize((128, 128))

In [40]:
img.size[0], img.size[1]

(352, 198)

In [42]:
int(str(img.size[0] / 32).split(".")[0])

11